In [1]:
import os
import tqdm
os.chdir("../")

In [2]:
from parser_FAR_dvf import *

In [3]:
prefix = "/home/cosmo/Documents/projects/bbchallenge/JEB/bbchallenge-deciders/decider-finite-automata-reduction/output"
dvf_JEB = FAR_DVF.from_file(prefix+"/finite_automata_reduction.dvf", pre_scan=True, compute_n_entries=True)
dvf_Tony = FAR_DVF.from_file(prefix+"/FAR_6_DFA_states.dvf", pre_scan=True)

## Extract machines solved

In [12]:
solved_JEB = set({})
with open(prefix+"/finite_automata_reduction.dvf", "rb") as f:
    for i in tqdm.tqdm(range(dvf_JEB.n_entries)):
        header, _ = dvf_JEB.ith_entry(f,i, ignore_n_entries=True)
        solved_JEB.add(header.machine_id)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 504210/504210 [00:04<00:00, 119532.08it/s]


In [13]:
solved_Tony = set({})
with open(prefix+"/FAR_6_DFA_states.dvf", "rb") as f:
    for i in tqdm.tqdm(range(dvf_Tony.n_entries)):
        header, _ = dvf_Tony.ith_entry(f, i)
        solved_Tony.add(header.machine_id)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503169/503169 [00:04<00:00, 113173.67it/s]


In [6]:
len(solved_JEB)

503169

In [7]:
len(solved_Tony)

503169

In [39]:
list(solved_Tony)[1]

5242882

In [8]:
solved_JEB == solved_Tony

True

In [29]:
a: int = 4
a.to_bytes(4,"big")

b'\x00\x00\x00\x04'

In [32]:
with open("FAR-finite-automata-reduction-JEB-max-6-DFA-states", "wb") as f:
    for machine_id in sorted(solved_Tony):
        f.write(machine_id.to_bytes(4,"big"))

## Extract DFAs

In [17]:
DFAs_JEB = {}
with open(prefix+"/finite_automata_reduction.dvf", "rb") as f:
    for i in tqdm.tqdm(range(dvf_JEB.n_entries)):
        header, entry = dvf_JEB.ith_entry(f,i)
        if header.machine_id not in DFAs_JEB:
            DFAs_JEB[header.machine_id] = []
        DFAs_JEB[header.machine_id].append(str(entry.dfa_transitions))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 504210/504210 [00:06<00:00, 77447.51it/s]


In [18]:
DFAs_Tony = {}
with open(prefix+"/FAR_6_DFA_states.dvf", "rb") as f:
    for i in tqdm.tqdm(range(dvf_Tony.n_entries)):
        header, entry = dvf_Tony.ith_entry(f,i)
        if header.machine_id not in DFAs_Tony:
            DFAs_Tony[header.machine_id] = []
        DFAs_Tony[header.machine_id].append(str(entry.dfa_transitions))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503169/503169 [00:06<00:00, 75697.15it/s]


In [19]:
for m in DFAs_Tony:
    if not DFAs_Tony[m][0] in DFAs_JEB[m]:
        print(m)
        print(DFAs_Tony[m][0])
        print(DFAs_JEB[m])
        break

75
[[0, 1], [2, 3], [1, 3], [3, 4], [3, 5], [3, 3]]
['[[0, 1], [2, 3], [1, 3], [3, 3]]']


In [20]:
count = 0
for m in DFAs_Tony:
    if not DFAs_Tony[m][0] in DFAs_JEB[m]:
        count += 1
count

465239

In [21]:
len(DFAs_Tony)

503169

In [23]:
len(DFAs_Tony[75][0])

48

In [24]:
for m in DFAs_Tony:
    if len(DFAs_Tony[m][0]) != 48:
        print(m)

In [35]:
def ithl(i):
    return chr(ord("A") + i)

def to_bbchallenge_format(tm) -> str:
    to_ret = ""
    for i, b in enumerate(tm):
        if i % 6 == 0 and i != 0:
            to_ret += "_"

        if i % 3 == 0 and tm[i + 2] == 0:
            to_ret += "-"
            continue
        if i % 3 == 1 and tm[i + 1] == 0:
            to_ret += "-"
            continue

        if i % 3 == 0:
            to_ret += "0" if b == 0 else "1"
        elif i % 3 == 1:
            to_ret += "R" if b == 0 else "L"
        else:
            if b == 0:
                to_ret += "-"
            else:
                to_ret += ithl(b - 1)

    return to_ret

In [33]:
from tm_utils import load_machine_from_db

In [37]:
with open("bb5_undecided_machines.csv", "w") as f:
    f.write("machine ID,machine description\n")
    with open("../all_5_states_undecided_machines_with_global_header", "rb") as fdb:
        with open("../bb5_undecided_index", "rb") as fund:
            raw_fund = fund.read()
            for i in range(0,len(raw_fund),4):
                machine_id = int.from_bytes(raw_fund[i:i+4],byteorder="big")
                machine_desc = to_bbchallenge_format(load_machine_from_db(fdb,machine_id))
                f.write(f"{machine_id},{machine_desc}\n")
                